In [7]:
import numpy as np
import pandas as pd


In [2]:
kanto = pd.read_csv('../data/kanto_stations_with_3clusters.csv')
kansai = pd.read_csv('../data/kansai_stations_with_cluster3.csv')
kanto_pca = pd.read_csv('../data/kanto_stations_pca.csv')
kansai_pca = pd.read_csv('../data/kansai_stations_pca.csv')

In [3]:
kanto_pca.head()

,pc1,pc2
0,-0.932476,0.728841
1,-1.186589,-0.635504
2,0.137028,-0.274607
3,0.468508,-1.378730
4,0.902938,-0.362098


In [4]:
kanto_stations = kanto.join(kanto_pca)
kanto_stations = kanto_stations.drop('Unnamed: 0', axis=1)
kanto_stations = kanto_stations.drop('Unnamed: 0.1', axis=1)
kanto_stations = kanto_stations.drop('station_name_roman', axis=1)

In [5]:
stations = pd.read_csv('../../../data/stations.csv')

roman = stations[['station_cd', 'station_name_roman']]

kanto_stations = pd.merge(kanto_stations, roman, on='station_cd', how='left')

In [6]:
len(kanto_stations)

2031

In [7]:
stations.columns

Index(['station_cd', 'station_g_cd', 'station_name', 'station_name_k',
       'station_name_r', 'line_cd', 'pref_cd', 'post', 'address', 'lon', 'lat',
       'open_ymd', 'close_ymd', 'e_status', 'e_sort', 'station_name_roman'],
      dtype='object')

In [8]:
kansai_stations = kansai.join(kansai_pca)

In [9]:
kanto_stations.columns

Index(['station_cd', 'station_g_cd', 'station_name', 'station_name_k',
       'station_name_r', 'line_cd', 'pref_cd', 'post', 'address', 'lon', 'lat',
       'open_ymd', 'close_ymd', 'e_status', 'e_sort', 'n', 'm', 'k_avg',
       'intersection_count', 'streets_per_node_avg', 'streets_per_node_counts',
       'streets_per_node_proportion', 'edge_length_total', 'edge_length_avg',
       'street_length_total', 'street_length_avg', 'street_segments_count',
       'node_density_km', 'intersection_density_km', 'edge_density_km',
       'street_density_km', 'circuity_avg', 'self_loop_proportion',
       'clean_intersection_count', 'clean_intersection_density_km',
       'cluster_number', 'pc1', 'pc2', 'station_name_roman'],
      dtype='object')

In [10]:
kanto_stations.to_csv('./kanto.csv', index=False)

In [11]:
kansai_stations.to_csv('./kansai.csv', index=False)

In [2]:
kanto_stations = pd.read_csv('./kanto.csv')
kansai_stations = pd.read_csv('./kansai.csv')

In [3]:
import folium
#center_lat, center_lon = (35.6809591, 139.7673068) # 東京駅
center_lat, center_lon = (34.6413, 135.5629) # 大阪の緯度経度

m = folium.Map(
    location=[center_lat, center_lon],
    zoom_start=12,
    width=800, 
    height=600, 
    tiles='openstreetmap') # マップ作成

folium.GeoJson('../data/kanto.geojson', name='region_name',
              style_function = lambda x: {
                                        'fillOpacity': 0.3,
                                        'fillColor': 'green' if x['properties']['use_area_type'] < 8 else 'orange' if x['properties']['use_area_type'] < 10 else 'gray' if x['properties']['use_area_type'] < 13 else 'red',
                                        'color': 'gray',
                                        'weight': 1,
                                        'popup': x['properties']['use_area_name']
              }).add_to(m)

folium.GeoJson('../data/kansai.geojson', name='region_name',
              style_function = lambda x: {
                                        'fillOpacity': 0.3,
                                        'fillColor': 'green' if x['properties']['use_area_type'] < 8 else 'orange' if x['properties']['use_area_type'] < 10 else 'gray' if x['properties']['use_area_type'] < 13 else 'red',
                                        'color': 'gray',
                                        'weight': 1,
                                        'popup': x['properties']['use_area_name']
              }).add_to(m)

folium.LayerControl().add_to(m) # 地図表示切替ボタン


In [4]:
for _, station in kanto_stations.iterrows():
    color = 'pink' if station['cluster_number'] == 0 else 'blue' if station['cluster_number'] == 1 else 'red' if station['cluster_number'] == 2 else 'green' if station['cluster_number'] == 3 else 'orange'
    folium.Marker([station.lat, station.lon], popup=str(station['station_name_roman']), icon=folium.Icon(color=color)).add_to(m)

In [5]:
for _, station in kansai_stations.iterrows():
    color = 'pink' if station['cluster_number'] == 0 else 'blue' if station['cluster_number'] == 1 else 'red' if station['cluster_number'] == 2 else 'green' if station['cluster_number'] == 3 else 'orange'
    folium.Marker([station.lat, station.lon], popup=str(station['station_name_roman']), icon=folium.Icon(color=color)).add_to(m)

In [ ]:
m

In [8]:
m.save('area_use_all2.html') # 保存